# Задание 1

In [72]:
import requests

response = requests.get("https://jsonplaceholder.typicode.com/posts")
data = response.json()
for i in range(0, 5):
    print(data[i]["title"], end = "\n")


sunt aut facere repellat provident occaecati excepturi optio reprehenderit
qui est esse
ea molestias quasi exercitationem repellat qui ipsa sit aut
eum et est occaecati
nesciunt quas odio


# Задание 2

In [71]:
city = input()
api_key = input()
url = f'http://api.openweathermap.org/geo/1.0/direct?q={city}&appid={api_key}'
city_response = requests.get(url)
data = city_response.json()

lat = data[0]["lat"]
lon = data[0]["lon"]

weather_url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}'
weather_response = requests.get(weather_url)
data = weather_response.json()
print(f'temp: {data["main"]["temp"]}, description: {data["weather"][0]["description"]}')

temp: 277.97, description: overcast clouds
